In [2]:
# Importamos la biblioteca NumPy para operaciones numéricas y matrices multidimensionales
import numpy as np 

# Importamos la biblioteca Pandas para análisis de datos tabulares
import pandas as pd 

# Importamos el módulo math para funciones matemáticas básicas
import math

# Importamos el módulo pyeeg para cálculos de características de señales EEG
import pyeeg as pe

# Importamos el módulo pickle para serialización de objetos Python
import pickle as pickle 

# Importamos el módulo os para interacción con el sistema operativo
import os 

# Importamos el módulo time para trabajar con medidas de tiempo
import time 


In [2]:
# Configuración para la función np.load para cargar archivos. 
# Los valores por defecto establecidos incluyen el uso de codificación ASCII.
np.load.__defaults__=(None, True, True, 'ASCII')

# Definición de los canales EEG que se desean utilizar (8 canales de openbci).
channel = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32]

# Frecuencias de las bandas para calcular el espectro de potencia (PSD) de la señal EEG.
band = [.5,4,8,12,30,45]  # Definición de 5 bandas.

# Tamaño de ventana para promediar la potencia en bandas durante 5 segundos (downsampleado a 128 Hz).
window_size = 128 * 5  # Ventana de 5 segundos a una frecuencia de muestreo de 128 Hz.

# Paso para actualizar el tiempo cada ciertos pasos.
step_size = 16  # Actualizar cada 0.125 segundos.

# Frecuencia de muestreo de la señal EEG.
sample_rate = 128  # Frecuencia de muestreo de 128 Hz.

# Lista de sujetos para cargar los datos.
subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']


In [3]:
# Definición de la función para procesar datos EEG
def FFT_Processing(sub, channel, band, window_size, step_size, sample_rate):
    # Lista para almacenar metadatos
    meta = []

    # Abrir los datos usando pickle para descomprimir
    with open('datos\s' + sub + '.dat', 'rb') as file:
        subject = pickle.load(file, encoding='latin1')  # Resuelve el problema de datos de Python 2 utilizando la codificación latin1

        for i in range(0, 40):
            # Para las 40 pruebas realizadas
            data = subject["data"][i] 
            labels = subject["labels"][i]
            start = 0

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = []  # Vector de metadatos para el análisis
                for j in channel:
                    X = data[j][start : start + window_size]  # Dividir los datos crudos en ventanas de 2 segundos, con un intervalo de 0.125 segundos
                    Y = pe.bin_power(X, band, sample_rate)  # FFT en 5 segundos del canal j, en secuencia de theta, alpha, beta baja, beta alta, gamma
                    meta_data = meta_data + list(Y[0])

                meta_array.append(np.array(meta_data))
                meta_array.append(labels)

                meta.append(np.array(meta_array))    
                start = start + step_size

        meta = np.array(meta)
        np.save('out\s' + sub, meta, allow_pickle=True, fix_imports=True)



In [4]:
for subjects in subjectList:
    FFT_Processing (subjects, channel, band, window_size, step_size, sample_rate)

<ipython-input-3-1f4853850be8>:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  meta.append(np.array(meta_array))


In [ ]:
# Listas para almacenar datos y etiquetas de los diferentes conjuntos
data_training = []
label_training = []
data_testing = []
label_testing = []
data_validation = []
label_validation = []

# Iteramos sobre la lista de sujetos
for subject in subjectList:
    # Cargamos los datos procesados para el sujeto actual
    with open(os.path.join('out', f's{subject}.npy'), 'rb') as file:
        sub = np.load(file)
        # Iteramos sobre los datos procesados del sujeto
        for i in range(sub.shape[0]):
            if i % 8 == 0:
                data_testing.append(sub[i][0])
                label_testing.append(sub[i][1])
            elif i % 8 == 1:
                data_validation.append(sub[i][0])
                label_validation.append(sub[i][1])
            else:
                data_training.append(sub[i][0])
                label_training.append(sub[i][1])

# Guardamos los conjuntos de datos y etiquetas en archivos
np.save(os.path.join('out', 'data_training.npy'), np.array(data_training), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'label_training.npy'), np.array(label_training), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'data_testing.npy'), np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'label_testing.npy'), np.array(label_testing), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'data_validation.npy'), np.array(data_validation), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'label_validation.npy'), np.array(label_validation), allow_pickle=True, fix_imports=True)

# Imprimimos las dimensiones de los conjuntos de datos y etiquetas para cada conjunto
print("Training dataset:", np.array(data_training).shape, np.array(label_training).shape)
print("Testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)
print("Validation dataset:", np.array(data_validation).shape, np.array(label_validation).shape)

# Restauramos los valores por defecto para la función np.load
np.load.__defaults__=(None, False, True, 'ASCII')


Training dataset: (445440, 160) (445440, 4)
Testing dataset: (74240, 160) (74240, 4)
Validation dataset: (74240, 160) (74240, 4)
